## lifelib interactions

Some interactions with the [lifelib BasicTerm_S](https://lifelib.io/libraries/basiclife/BasicTerm_S.html) actuarial cashflow model.

***Run cells in sequence (see Play button), and use sliders to interact***

In [ ]:
%pip install openpyxl

In [ ]:
import pandas
from BasicTerm_S_nomx import BasicTerm_S

In [ ]:
%pip install ipywidgets

In [ ]:
from ipywidgets import interact

## cashflow interaction (visual)

In [ ]:
%pip install altair

In [ ]:
import altair as alt
import numpy as np

In [ ]:
def interactive_cashflows(p=1):
    df = BasicTerm_S.Projection[p].result_cf().reset_index().melt(id_vars=['index'], var_name='cashflow')
    df['value'] = np.where(df['cashflow'] != 'Premiums', df['value']*-1,df['value'])
    df['value'] = np.where(df['cashflow'] == 'Net Cashflow', df['value']*-1,df['value'])
    #df = df[df['cashflow'] != 'Net Cashflow']

    # Vega-Altair layered chart
    
    base = alt.Chart(df).properties(width=580, height=400)

    bars = base.mark_bar().transform_filter(filter="datum.cashflow != 'Net Cashflow'").encode(
            tooltip=['cashflow:N','value:Q','index:N'],
            x=alt.X('index:Q', scale=alt.Scale(domain=[0,200]), title="time"),
            y=alt.Y('value:Q', scale=alt.Scale(domain=[-500,300])),
            color='cashflow:N'
          )

    lines = base.mark_line(interpolate='step-after').transform_filter(filter="datum.cashflow == 'Net Cashflow'").encode(
            tooltip=['key:N','value:Q','index:N'],
            x=alt.X('index:Q', scale=alt.Scale(domain=[0,200]), title="time"),
            y=alt.Y('value:Q', scale=alt.Scale(domain=[-500,300])),
            color=alt.value('black')
          )

    return bars+lines

In [ ]:
interact(interactive_cashflows, p=(1,10000))

## summary interaction

In [ ]:
interact(lambda p: BasicTerm_S.Projection[p].result_pv(), p=(1,10000))

## cashflow interaction

In [ ]:
interact(lambda p: BasicTerm_S.Projection[p].result_cf(), p=(1,10000))

.

.

.

.

.

.

.

.

.

## all-policies 🚧 WIP

In [ ]:
[BasicTerm_S.Projection[p].result_pv().assign(p=p) for p in [1,2,3]][2]

In [ ]:
from functools import reduce

In [ ]:
#40s runtime: pandas.concat([BasicTerm_S.Projection[p].result_pv()[:1].assign(p=p) for p in range(1,10000) ])

In [ ]:
#0.1s runtime (local):
policy_results = pandas.DataFrame([ ( p, BasicTerm_S.Projection[p].age_at_entry(), BasicTerm_S.Projection[p].sex(), BasicTerm_S.Projection[p].policy_term(), BasicTerm_S.Projection[p].sum_assured(), BasicTerm_S.Projection[p].pv_net_cf(), ) for p in range(1,10000) ], columns=['p','age_at_entry','sex','policy_term','sum_assured','pv_net_cf'])

In [ ]:
policy_results

In [ ]:
alt.data_transformers.disable_max_rows()

alt.Chart(policy_results).properties(width=480, height=200, title='BasicTerm Profitability by Policy (@ outset; 10k points)').mark_point().encode(
            x='sum_assured:Q',
            y='pv_net_cf:Q',
            #color='age_at_entry:Q',
            color=alt.Color('age_at_entry:Q').scale(scheme='spectral'),
            shape='sex:N',
            row='policy_term:N',
            order='p'
          )